# Pipeline final pour l'entreprise **TouNum**

L'entreprise TouNum travaille sur la numérisation de documents (textes, images...). Leurs services sont souvent requis par des entreprises numérisant leur base de documents papier. Ils souhaitent étendre leur gamme de services pour inclure des outils de Machine Learning. En effet, certains de leurs clients ont une grande quantité de données à numériser, et un service de catégorisation automatique serait plus que valorisable.

Cette pipeline sert donc à remplir l'objectif de TouNum, pour cela elle est divisée en 3 grande partie :
1. Classification d'image selon 5 catégories [Photo, Peinture, Schéma, Croquis, Texte]

2. Débruitage des images classifiées "Photo", suppresion de 4 types de bruits : [Gaussian, Poisson, Speckles, Salt & pepper]
   
3. Génération des descriptions pour les photos débruitées.

Format des données d'entrées attendues :
- **Chemin des données d'entrée :** `datas_path` (actuellement défini comme `./Dataset/test_final`)
- **Exemple de fichiers attendus dans le dossier :**
    - `datas_path/img01.png`
    - `datas_path/img02.jpg`
    - ...


---
## Dépendances

Chargement des différentes dépendances nécessaires au bon fonctionnement de la pipeline, ainsi que de la configuration de la pipeline.

In [ ]:
# Import des dépendances
import os
import sys

# Ajouter le dossier courant au path si nécessaire pour accéder à notre librairies utils.py
sys.path.append(os.getcwd())
import utils

In [ ]:
# Configuration Principale
datas_path = "./Dataset/test_final" # Dossier de stockage des données qui sont des images
autoencoder_weights_path = "./checkpoints/modelgeneric_epoch21_valLoss0.0758.h5" # Chemin du stockage des poids de l'autoencoder


---
## Partie 1 : Classification de photo (Livrable 1)

Dans cette partie, nous allons charger notre modèle de classification d'images capable de distinguer entre 5 catégories : [Photo, Peinture, Schéma, Croquis, Texte]. 

Pour ce faire, nous allons suivre les étapes suivantes :
1. Préparation des données : collecte, nettoyage et augmentation des données pour chaque catégorie.
2. Chargement du modèle : utilisation d'un réseau de neurones convolutionnel (CNN) pour effectuer la classification.
3. Classification des données réelles : Classification des images fournies par l'entreprise TouNum. 
4. Visualisation des résultats : Visualisations de plusieurs résultats de notre modèle.

L'objectif est de fournir un modèle robuste et performant qui pourra être intégré dans la pipeline globale de l'entreprise.

### 1.1 Préparations des données
Chargement et nettoyage de celle-ci.

In [ ]:
# # A SUPPRIMER AAVNT DE METTRE EN PROD


# # Chargement des données 
# import os
# import shutil
# import random

# # === CONFIGURATION ===
# X = 1000  # Nombre total d'images à copier (modifie selon ton besoin)
# source_base = "./Dataset/Dataset1"
# target_dir = "./Dataset/test_final"

# # Extensions autorisées (images)
# allowed_extensions = {".jpg", ".jpeg", ".png", ".bmp", ".tiff", ".gif"}

# # Créer le dossier cible s’il n’existe pas
# os.makedirs(target_dir, exist_ok=True)

# # Récupération de toutes les images valides dans les sous-dossiers
# all_images = []
# for folder in ["Photo", "Painting", "Sketch", "Text", "Schematics"]:
#     folder_path = os.path.join(source_base, folder)
#     for filename in os.listdir(folder_path):
#         if any(filename.lower().endswith(ext) for ext in allowed_extensions):
#             full_path = os.path.join(folder_path, filename)
#             all_images.append(full_path)

# # Vérification que X n'est pas trop grand
# if X > len(all_images):
#     raise ValueError(f"Trop d’images demandées ({X}), seulement {len(all_images)} disponibles.")

# # Sélection aléatoire sans doublons
# selected_images = random.sample(all_images, X)

# # Copie des fichiers
# for img_path in selected_images:
#     filename = os.path.basename(img_path)
#     dest_path = os.path.join(target_dir, filename)
    
#     # Pour éviter les doublons de nom de fichier, on peut renommer si nécessaire
#     if os.path.exists(dest_path):
#         name, ext = os.path.splitext(filename)
#         count = 1
#         while os.path.exists(dest_path):
#             filename = f"{name}_{count}{ext}"
#             dest_path = os.path.join(target_dir, filename)
#             count += 1

#     shutil.copy(img_path, dest_path)

# print(f"✅ {X} images copiées dans {target_dir}")


In [ ]:
# Affichages des statistiques des images
images_count = utils.get_infos_datas(datas_path)

In [ ]:
# Affichage de quelques images
utils.display_random_images(datas_path,5)

In [ ]:
# Nettoyage des données
# utils.clean_images_dataset(datas_path)

In [ ]:
# Redimensionnement des images en 256, 256
# utils.resize_images_dataset(datas_path, (256, 256))

# Affichage de quelques images après redimensionnement
utils.display_random_images(datas_path,5)


### 1.2 Chargement du modèle
Chargement de notre modèle précèdement entrainé.

In [ ]:
# Chargement de notre modèle le plus performant ()
model = utils.load_model("./Backup model/Livrable 1/model_Categorical_modified2.h5")

### 1.3 Classification des données
Classification des images selon les 5 catégories ()

In [ ]:
# Classification des images (stocker les images de types "Photo")
list_photos = utils.get_predicted_photos(model, datas_path)

### 1.4 Affichage de quelques résultats

In [ ]:
# Affichage de quelques images après classification
utils.display_images_with_predictions(model, datas_path, 5)

---
## Partie 2 : Débruitage de nos images (Livrable 2)

Dans cette partie, nous allons utiliser notre modèle de débruitage d'images pour les images classifiées comme "Photo". L'objectif est de supprimer efficacement les bruits présents dans les images tout en préservant leurs détails importants. Les étapes suivantes seront suivies :

1. Chargment de notre autoencoder : chargemnt de notre autoencoder le plus performant. 
2. Débruitage de nos images : débruitages des images catégorisées photos, grâce à notre model de classification.
3. Visualtion des résulats : affichage d'échantillons de nos images débruitées.

L'objectif est de fournir des images de haute qualité prêtes pour l'étape suivante de la pipeline.

### 2.1 Chargement de l'autoencoder
On charge notre autoencoder le plus performant. (générique)

In [ ]:
autoencoder = utils.load_autoencoder(autoencoder_weights_path)

### 2.2 Débruitage des images

In [ ]:
# Charger les images à partir des chemins
loaded_images = utils.load_images_from_paths(list_photos)
loaded_images = utils.resize_images(loaded_images, (256, 256))
loaded_images = utils.preprocess_data(loaded_images)

images_denoised = utils.denoise_images(autoencoder, loaded_images)

### 2.3 Résultats de débruitage
Affichage d'échantillons d'images denoised

In [ ]:
utils.plot_denoised_examples(loaded_images, images_denoised)

## Partie 3 : Captionning d'images (Livrable 3)
Dans cette partie, nous allons utiliser un modèle de génération de descriptions (captionning) pour les images débruitées. L'objectif est de produire des descriptions textuelles pertinentes pour chaque image, facilitant ainsi leur compréhension et leur utilisation. Les étapes suivantes seront suivies :

1. Chargement du modèle de captionning : utilisation d'un modèle pré-entraîné pour la génération de descriptions.
2. Génération des descriptions : application du modèle sur les images débruitées pour produire des descriptions textuelles.
3. Visualisation des résultats : affichage des descriptions générées pour un échantillon d'images.

### 3.1 Chargement du modèle de captionning

### 3.2 Captionning des images
Ajout d'une descriptions pour les images

### 3.3 Résultats du captionning
Affichage des descriptions générées pour les images.